In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
input_file = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\IT Services & Consulting"

In [ ]:
company_files = os.listdir(input_file)

for company in company_files:
    folder_path = f"{input_file}/{company}"
    
    company_folders = os.listdir(folder_path)
    
    for folders in company_folders:
        print(folders)
    
    pass

company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
company_params.json
Excel
Pruned_Excel
IT Services & Consulting_Common_Parameters.txt
Pruned_Excel


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\sharm\\OneDrive\\Desktop\\Kishan\\Contractzy\\WebScrapping\\Tutorial\\Financial_Data\\MoneyControl\\Companies\\IT Services & Consulting/common_sector_param.json'